# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 - Vandana Sridhar
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
#citations.show(5)

In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
#patents.show(5)

> Selected just the patent number and the patent state to obtain the modified patent table

In [8]:
patent_modified = patents.select('PATENT','POSTATE')

In [9]:
#patent_modified.show()

> Provided an alias name for the citations and the modified patents table

In [10]:
p = patent_modified.alias('p')
c= citations.alias('c')

> Created an inner join between the citations and the modified patent table based on cited number

In [11]:
join1 = c.join(p,c.CITED == p.PATENT, 'inner')
#join1.show()

> Selected the citing number, cited number and the cited state from the join1 resulting table

In [12]:
new_table = join1.select('CITING','CITED',col('POSTATE').alias("CITED_STATE"))
#new_table.show()

> Made another join between the above table and patents based on citing number to obtain the citing state

In [13]:
join2 = new_table.join(p,new_table.CITING == p.PATENT, 'inner')
#join2.show()


> Dropped the patent column for filtering purposes

In [14]:
it = join2.drop('PATENT')
#intermediate_table.show()

> After dropping patent column, the intermediate table is obtained with citing number, citing state, cited number and cited state

In [15]:
intermediate_table = it.select('CITING',col('POSTATE').alias('CITING_STATE'),'CITED','CITED_STATE')


In [16]:
#intermediate_table.show()

> Filtered out rows that have same states citations alone from the intermediate table

In [17]:
same_state = intermediate_table.where(intermediate_table.CITING_STATE == intermediate_table.CITED_STATE)

In [18]:
#same_state.show()

> To obtain the number of same state citations for the citing number, the table is grouped by the citing number which is the key and the count of the same state citations is obtained and aggregated using the below statement. I have created a separate column named count for this purpose.

In [19]:
count_same_state = same_state.groupby('CITING').agg(f.count(same_state.CITING_STATE == same_state.CITED_STATE).alias('count'))

In [20]:
#count_same_state.show()

> To obtain the highest count, I have used the ordered the count in descending order.

In [21]:
final_count = count_same_state.orderBy('count',ascending=False)

> Final counts are displayed

In [22]:
final_count.show()

+-------+-----+
| CITING|count|
+-------+-----+
|5959466|  125|
|5983822|  103|
|6008204|  100|
|5952345|   98|
|5958954|   96|
|5998655|   96|
|5936426|   94|
|5980517|   90|
|5913855|   90|
|5739256|   90|
|5951547|   90|
|5925042|   90|
|5978329|   90|
|5978331|   89|
|5689485|   89|
|5928229|   89|
|5618907|   89|
|5917082|   89|
|5602226|   88|
|5847160|   87|
+-------+-----+
only showing top 20 rows



> Now to append the required count to the patents table, I have done an inner join with the final count and the patent's table

In [23]:
final_output = patents.join(final_count, patents.PATENT == final_count.CITING, 'inner')

In [24]:
#final_output.show()

> To obtain the top 10 patents that have the most self-state citations in descending order

In [25]:
final_samestate = final_output.orderBy('count',ascending=False).show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|5959466|  125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|5983822|  103|
|6008204| 1999|